<a href="https://colab.research.google.com/github/lpioltino/Analitica-Predictiva/blob/main/Modelos_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn import preprocessing
from sklearn.metrics import r2_score
from sklearn.preprocessing import scale
from math import *
from sklearn import datasets, linear_model

In [ ]:
data =pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/base.csv',sep=',')

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:

plt.figure(figsize =(24,12))
sns.heatmap(data.corr(), annot = True)

Elimino la variable worktype children porque con la edad es suficiente. Son dos variables que dicen lo mismo de otra manera 

In [ ]:
del data['work_type_children'] 

PRUEBO CON TODAS LAS VARIABLES 

In [ ]:
y=data['stroke']

In [ ]:
y.value_counts()

In [ ]:
x=data[['age','hypertension','heart_disease','avg_glucose_level','bmi','gender_Female','gender_Male','gender_Other','ever_married_No','ever_married_Yes','work_type_Govt_job','work_type_Never_worked','work_type_Private','work_type_Self-employed','Residence_type_Rural','Residence_type_Urban','smoking_status_formerly smoked','smoking_status_never smoked','smoking_status_smokes','smoking_status_Unknown']]

BALANCEO DE CLASES 

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=27)
x,y= sm.fit_resample(x, y)

In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 0 )

SACO LAS VARIABLES NO SIGNIFICATIVAS

In [ ]:
y2=data['stroke']

In [ ]:
x2=data[['age','hypertension','heart_disease','avg_glucose_level','bmi','ever_married_No','ever_married_Yes','work_type_Govt_job','work_type_Never_worked','work_type_Private','work_type_Self-employed','smoking_status_formerly smoked','smoking_status_never smoked','smoking_status_smokes','smoking_status_Unknown']]

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=27)
x2,y2= sm.fit_resample(x2, y2)

In [ ]:
from sklearn.model_selection import train_test_split 
x2_train, x2_test, y2_train,y2_test = train_test_split(x2,y,test_size = 0.3,random_state = 0 )

In [ ]:
print("Training Data: {}".format(len(x_train)))
print("Testing Data: {}".format(len(x_test)))

MODELOS

RANDOM FOREST

MODELO 1 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier(max_depth=18, random_state= 42)
random_forest_model.fit(x_train, y_train)

RandomForestClassifier(max_depth=18, random_state=42)

In [ ]:
random_forest_model.score(x_test, y_test)

0.9636612958519026

In [ ]:
random_forest_model.score(x_train, y_train)

0.9997060984570169

In [ ]:
fila = []

for i in range(14,29,2):
  print(i)
  rf = RandomForestClassifier(max_depth=i,random_state=42)
  rf.fit(x_train,y_train)
  r2_train = rf.score(x_train, y_train)
  r2_test = rf.score(x_test, y_test)
  fila.append([i,r2_train,r2_test])

In [ ]:
scores_prof = pd.DataFrame(fila, columns=["max_depth","r2_train","r2_test"])
scores_prof

MATRIZ

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(random_forest_model, x_test,y_test,cmap='Blues')

In [ ]:
sorted_idx = random_forest_model.feature_importances_.argsort()
plt.barh(x_train.columns[sorted_idx], random_forest_model.feature_importances_[sorted_idx])
plt.xlabel("Random Forest Feature Importance")

In [ ]:
y_predR = random_forest_model.predict(x_test)
from sklearn.metrics import roc_auc_score
from sklearn import metrics
print("Accuracy Score: {}".format(accuracy_score(y_test,y_predR)))
print("Precision:",metrics.precision_score(y_test, y_predR))
print("Recall:",metrics.recall_score(y_test, y_predR))


roc_auc_score(y_test, random_forest_model.predict_proba(x_test)[:,1])

In [ ]:
y_pred_logreg_proba = random_forest_model.predict_proba(x_test)
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_logreg_proba[:,1])
plt.figure(figsize=(6,4))
plt.plot(fpr,tpr,'-g',linewidth=1)
plt.plot([0,1], [0,1], 'k--' )
plt.title('ROC curve for Random Forest Model')
plt.xlabel("False Positive Rate")
plt.ylabel('True Positive Rate')
plt.show()

MODELO 2

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest_model2 = RandomForestClassifier(max_depth=18, random_state= 42)
random_forest_model2.fit(x2_train, y2_train)

In [ ]:
random_forest_model2.score(x2_test, y2_test)

In [ ]:
random_forest_model2.score(x2_train, y2_train)

In [ ]:
fila2 = []

for i in range(14,29,2):
  print(i)
  rf2 = RandomForestClassifier(max_depth=i,random_state=42)
  rf2.fit(x2_train,y2_train)
  r22_train = rf2.score(x2_train, y2_train)
  r22_test = rf2.score(x2_test, y2_test)
  fila2.append([i,r22_train,r22_test])

In [ ]:
scores_prof2 = pd.DataFrame(fila2, columns=["max_depth","r2_train","r2_test"])
scores_prof2

XG BOOST

In [ ]:
import xgboost as xgb
XGboost = xgb.XGBClassifier(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.2,
                max_depth = 22, alpha = 10, n_estimators = 150)
XGboost.fit(x_train,y_train)


In [ ]:
predsXG = XGboost.predict(x_test)
r2_score(y_test,predsXG)

In [ ]:
predsXGt = XGboost.predict(x_train)
r2_score(y_train,predsXGt)

In [ ]:
fila2 = []

for i in range(14,29,2):
  xb = xgb.XGBClassifier(max_depth=i)
  xb.fit(x_train,y_train)
  r2_train = xb.score(x_train, y_train)
  r2_test = xb.score(x_test, y_test)
  fila2.append([i,r2_train,r2_test])

In [ ]:
scores_depth_xb = pd.DataFrame(fila2, columns=["max_depth","r2_train","r2_test"])
scores_depth_xb

In [ ]:
xgb.plot_importance(XGboost)
plt.rcParams['figure.figsize'] = [8, 8]
plt.show()

In [ ]:
xgb.plot_tree(XGboost,num_trees=0)
plt.rcParams['figure.figsize'] = [10,10]
plt.show()

In [ ]:
predsXG = XGboost.predict(x_test)
from sklearn.metrics import roc_auc_score
from sklearn import metrics
print("Accuracy Score: {}".format(accuracy_score(y_test,predsXG)))
print("Precision:",metrics.precision_score(y_test, predsXG))
print("Recall:",metrics.recall_score(y_test,predsXG))


roc_auc_score(y_test,  XGboost.predict_proba(x_test)[:,1])

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(XGboost, x_test,y_test,cmap='Blues')

REGRESION LOGÍSTICA 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.weightstats import ttest_ind
modelo = LogisticRegression(penalty='none')
modelo.fit(x_train,y_train)
y_pred=modelo.predict(x_test)

MATRIZ DE CONFUSION

In [ ]:
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap='Blues',fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

MEDIDAS

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))
roc_auc_score(y_test,  modelo.predict_proba(x_test)[:,1])

Accuracy: 0.9221803222488858
Precision: 0.9366197183098591
Recall: 0.9066121336059987


0.9783466140140564

CURVA ROC 

In [ ]:
y_pred_proba = modelo.predict_proba(x_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

RED NEURONAL

In [ ]:
from sklearn.neural_network import MLPClassifier
clf_red = MLPClassifier(hidden_layer_sizes=(7,), random_state=42)
clf_red.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


MLPClassifier(hidden_layer_sizes=(7,), random_state=42)

In [ ]:
clf_red.score(x_test,y_test)

In [ ]:
clf_red.score(x_train,y_train)

In [ ]:
plot_confusion_matrix(clf_red, x_test,y_test,cmap='Blues')

In [ ]:
red_pred = clf_red.predict(x_test)
print("Accuracy:",metrics.accuracy_score(y_test, red_pred))
print("Precision:",metrics.precision_score(y_test, red_pred))
print("Recall:",metrics.recall_score(y_test, red_pred))
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn import metrics
roc_auc_score(y_test, clf_red.predict_proba(x_test)[:,1])

In [ ]:
y_pred_red_proba = clf_red.predict_proba(x_test)
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_red_proba[:,1])
plt.figure(figsize=(6,4))
plt.plot(fpr,tpr,'-g',linewidth=1)
plt.plot([0,1], [0,1], 'k--' )
plt.title('ROC curve for Logistic Regression Model')
plt.xlabel("False Positive Rate")
plt.ylabel('True Positive Rate')
plt.show()